Welcome! We will be working on Web Scrapping and using APIs to get the neccesary data to analyse the Prices and Sizes for Rooms in the Copenhagen Metropolitan Area. 

First I import the necessary libraries and get the url data.

In [2]:
from bs4 import BeautifulSoup as soup
from pip._vendor import requests
import pandas as pd
url = "https://www.boligportal.dk/en/rental-apartments/k%C3%B8benhavn"

# get the contents of the webpage in text format and store in a variable called data
data = requests.get(url)
print(data)


<Response [200]>


The response 200 indicate us that we could acces the data. Let´s check that we get the info as HTML.

In [3]:
#We check that we get the HTML code from the webpage creatins a BeautifulSoup object
bsobj=soup(data.content, "lxml")
bsobj

<!DOCTYPE html>
<html>
<head>
<script data-culture="EN" id="CookieConsent" src="https://policy.app.cookieinformation.com/uc.js" type="text/javascript"></script>
<script>
      const modernBrowser = "fetch" in window && "assign" in Object;
      const isSafari = /constructor/i.test(window.HTMLElement) || (function (p) { return p.toString() === "[object SafariRemoteNotification]"; })(!window['safari'] || (typeof safari !== 'undefined' && window['safari'].pushNotification));
      const isIE11 = /*@cc_on!@*/false || !!document.documentMode;

      if (!modernBrowser || (isSafari || isIE11)) {
      const scriptElement = document.createElement("script");

      scriptElement.async = false;
      scriptElement.src = "https://dnejt4xibfee2.cloudfront.net/static/dist/polyfills.bundle.48224a6ce69e.js";
      document.head.appendChild(scriptElement);
      }
    </script>
<script>
      function loadZendeskWidget(key, onCompleteCallback) {
        const scriptElement = document.createElement("s

As we can see, I got the HTML code with some JavaScript code in it, and not JSON format.
The next step is to scrap the prices, areas and other relevant data into a pandas dataframe by crawling through the pages the get the 1739 rows.

In [4]:
#lets create the lists that will be loaded into the dataframe 


price_list = []
location_list = []
size_list =[]

#get the first rows of data

url = "https://www.boligportal.dk/en/rental-apartments/k%C3%B8benhavn/"

response = requests.get(url)
data = response.content
sopa = soup(data, "lxml")
appartments = sopa.find_all('div', {'class': 'css-1jf5j4m'})
#lets go through the rows and add the values to the aproppiate lists
for raw in appartments:
    for price in sopa.find_all('span', {'class': 'css-1wltohh'}):
        price_list.append(price.text)
    for location in sopa.find_all('div', {'class': 'css-22506a'}):
        location_list.append(location.text)
    for size in sopa.find_all('div', {'class': 'css-5oox4j'}):
        size_list.append(size.text)
    break   

# but we need to go through the whole data set, so we have to crawl through the pages, that advance by 18 new items 

add = 18
while add <= 1730:

    url = "https://www.boligportal.dk/en/rental-apartments/k%C3%B8benhavn/?offset="+str(add)
#this will allow us to go to the next page and the following one once the loop is completed
    response = requests.get(url)
    data = response.content
    sopa = soup(data, "lxml")
    appartments = sopa.find_all('div', {'class': 'css-1jf5j4m'})
    for raw in appartments:
        for price in sopa.find_all('span', {'class': 'css-1wltohh'}):
            price_list.append(price.text)
        for location in sopa.find_all('div', {'class': 'css-22506a'}):
            location_list.append(location.text)
        for size in sopa.find_all('div', {'class': 'css-5oox4j'}):
            size_list.append(size.text)
        break

    add=add+18


price_list, location_list, size_list






(['12.800\xa0kr.',
  '15.000\xa0kr.',
  '10.900\xa0kr.',
  '23.000\xa0kr.',
  '13.000\xa0kr.',
  '20.500\xa0kr.',
  '18.100\xa0kr.',
  '8.800\xa0kr.',
  '15.295\xa0kr.',
  '26.000\xa0kr.',
  '9.900\xa0kr.',
  '21.700\xa0kr.',
  '17.250\xa0kr.',
  '8.995\xa0kr.',
  '7.150\xa0kr.',
  '28.310\xa0kr.',
  '15.253\xa0kr.',
  '14.900\xa0kr.',
  '7.700\xa0kr.',
  '22.500\xa0kr.',
  '12.900\xa0kr.',
  '13.000\xa0kr.',
  '14.600\xa0kr.',
  '14.200\xa0kr.',
  '13.900\xa0kr.',
  '14.400\xa0kr.',
  '12.000\xa0kr.',
  '13.995\xa0kr.',
  '12.200\xa0kr.',
  '13.000\xa0kr.',
  '8.600\xa0kr.',
  '12.250\xa0kr.',
  '16.623\xa0kr.',
  '10.800\xa0kr.',
  '9.800\xa0kr.',
  '12.944\xa0kr.',
  '14.600\xa0kr.',
  '14.000\xa0kr.',
  '8.900\xa0kr.',
  '8.600\xa0kr.',
  '8.600\xa0kr.',
  '11.000\xa0kr.',
  '22.068\xa0kr.',
  '16.500\xa0kr.',
  '25.000\xa0kr.',
  '12.300\xa0kr.',
  '14.500\xa0kr.',
  '21.500\xa0kr.',
  '20.000\xa0kr.',
  '6.900\xa0kr.',
  '16.600\xa0kr.',
  '11.800\xa0kr.',
  '18.752\xa0kr.',
  '1

In [5]:
# zip those list into a pandas dataframe
import pandas as pd
data_tuples = list(zip(price_list, location_list, size_list))
df = pd.DataFrame(data_tuples, columns=['Price', 'Location',"Size"])
df

,Price,Location,Size
0,12.800 kr.,"København, Eremitageparken",3 rm. · Apartment · 84 m²
1,15.000 kr.,"København, Nordre Fasanvej",3 rm. · Apartment · 72 m²
2,10.900 kr.,"København, Peter Ipsens Allé",2 rm. · Apartment · 55 m²
3,23.000 kr.,"København, Johannevej",5 rm. · Apartment · 204 m²
4,13.000 kr.,"København, Storegårdsvej",3 rm. · Apartment · 88 m²
...,...,...,...
1580,40.000 kr.,"København, Arkonagade",3 rm. · Apartment · 125 m²
1581,5.200 kr.,"København, Jahnsensvej",1 rm. · Apartment · 27 m²
1582,9.000 kr.,"København, Brydes Allé",2 rm. · Apartment · 68 m²
1583,5.600 kr.,"København, Kulbanevej",1 rm. · Apartment · 38 m²


Now that we have the dataframe we have to wrangle the data to be able to work efficiently with it.

Lets first split the size column and eliminate the "Apartment" of the string because of redundancy.

In [6]:
df[['Rooms', 'Apartment', 'Apartment in m²']] = df['Size'].str.split(' · ', expand=True)
df.drop('Apartment', inplace=True, axis=1)
df.drop('Size', inplace=True, axis=1)

df.head()


,Price,Location,Rooms,Apartment in m²
0,12.800 kr.,"København, Eremitageparken",3 rm.,84 m²
1,15.000 kr.,"København, Nordre Fasanvej",3 rm.,72 m²
2,10.900 kr.,"København, Peter Ipsens Allé",2 rm.,55 m²
3,23.000 kr.,"København, Johannevej",5 rm.,204 m²
4,13.000 kr.,"København, Storegårdsvej",3 rm.,88 m²


DISCLAIMER

(After exporting the Excel File, cleaning and preparing it up, I found out that the locations were not recognizable for Tablau, so getting the ZIP codes of them was neccesary. That means, back at the lab.)

Now for visualisation pourpuses we need to get the zip code from each location to generate a map visualisation.

In [8]:
#import necesary libraries
from geopy.geocoders import Nominatim

#I create an empty Zip column
df["Zip"] = None


Time to use the geopy to get those zip codes. You may need to run the code a couple more times until the "Zip" column is fully populated.

In [ ]:
#I create a nominatim object
geo_locator = Nominatim(user_agent="geoapiExercises")
#get the complete address by using geocode() function in a loop, getting only the zip value and appending the values to the new zip column
df = df.reset_index(drop=True)
for index, row in df.iterrows():
    if(row["Zip"]==None):
        #gets the location with zipcode for the row
        place_1 = row["Location"]
        location = geo_locator.geocode(place_1)
        data_1 = location.raw
        #split to get only the zip code and append it to the zipcolumn
        location_data = data_1['display_name'].split()
        df.loc[index, "Zip"] = location_data[-2]
        print(row["Location"] + "=" + df.loc[index, "Zip"])


Now we can export the df as an Excel Notebook to further clean the data and get and visualise our results!



In [7]:
# writing to Excel
datatoexcel = r'C:\Users\sanri\Desktop\Proyecto WebScrapping\Copenhaguen Apartment Prices.xlsx'

# saving the excel

df.to_excel(datatoexcel, encoding='utf-8')
print('DataFrame is written to Excel File successfully.')


DataFrame is written to Excel File successfully.


Thanks for sticking around and please check the results!